# OceanbaseVectorStore

本指南介绍如何开始使用 Oceanbase vector store。

## 设置

要访问 OceanBase 向量存储，您需要部署一个独立的 OceanBase 服务器：

并安装 `langchain-oceanbase` 集成包。

检查与 OceanBase 的连接并为向量数据设置内存使用率：

In [14]:
from pyobvector import ObVecClient

tmp_client = ObVecClient()
tmp_client.perform_raw_text_sql("ALTER SYSTEM ob_vector_memory_limit_percentage = 30")

## 初始化

配置嵌入模型的 API 密钥。这里我们以 `DashScopeEmbeddings` 为例。当按照上述描述使用 Docker 镜像部署 `Oceanbase` 时，只需按照下面的脚本设置 `host`、`port`、`user`、`password` 和 `database name`。对于其他部署方式，请根据实际情况设置这些参数。

In [15]:
import os

from langchain_community.embeddings import DashScopeEmbeddings
from langchain_oceanbase.vectorstores import OceanbaseVectorStore

DASHSCOPE_API = os.environ.get("DASHSCOPE_API_KEY", "")
connection_args = {
    "host": "127.0.0.1",
    "port": "2881",
    "user": "root@test",
    "password": "",
    "db_name": "test",
}

embeddings = DashScopeEmbeddings(
    model="text-embedding-v1", dashscope_api_key=DASHSCOPE_API
)

vector_store = OceanbaseVectorStore(
    embedding_function=embeddings,
    table_name="langchain_vector",
    connection_args=connection_args,
    vidx_metric_type="l2",
    drop_old=True,
)

## 管理向量存储

### 将条目添加到向量存储

- TODO: 编辑然后运行代码单元以生成输出

In [16]:
from langchain_core.documents import Document

document_1 = Document(page_content="foo", metadata={"source": "https://foo.com"})
document_2 = Document(page_content="bar", metadata={"source": "https://bar.com"})
document_3 = Document(page_content="baz", metadata={"source": "https://baz.com"})

documents = [document_1, document_2, document_3]

vector_store.add_documents(documents=documents, ids=["1", "2", "3"])

['1', '2', '3']

### 更新向量存储中的条目

In [17]:
updated_document = Document(
    page_content="qux", metadata={"source": "https://another-example.com"}
)

vector_store.add_documents(documents=[updated_document], ids=["1"])

['1']

### 从向量存储中删除项目

In [18]:
vector_store.delete(ids=["3"])

## 查询向量存储

一旦你的向量存储创建完成并且已经添加了相关文档，在链（chain）或代理（agent）运行期间，你很可能希望对其进行查询。

### 直接查询

进行简单的相似性搜索可以这样做：

In [19]:
results = vector_store.similarity_search(
    query="thud", k=1, filter={"source": "https://another-example.com"}
)
for doc in results:
    print(f"* {doc.page_content} [{doc.metadata}]")

* bar [{'source': 'https://bar.com'}]


如果你想执行相似性搜索并获得相应的得分，可以运行：

In [20]:
results = vector_store.similarity_search_with_score(
    query="thud", k=1, filter={"source": "https://example.com"}
)
for doc, score in results:
    print(f"* [SIM={score:3f}] {doc.page_content} [{doc.metadata}]")

* [SIM=133.452299] bar [{'source': 'https://bar.com'}]


### 查询转换为检索器

你也可以将向量存储转换为检索器，以便在你的链中更方便地使用。

In [21]:
retriever = vector_store.as_retriever(search_kwargs={"k": 1})
retriever.invoke("thud")

[Document(metadata={'source': 'https://bar.com'}, page_content='bar')]

## 用于检索增强生成的使用方法

有关如何将此向量存储用于检索增强生成 (RAG) 的指南，请参阅以下章节：

- [教程](/docs/tutorials/)
- [操作指南：使用 RAG 进行问答](https://python.langchain.com/docs/how_to/#qa-with-rag)
- [检索概念文档](https://python.langchain.com/docs/concepts/#retrieval)

## API 参考

有关 OceanbaseVectorStore 的所有功能和配置的详细文档，请访问 API 参考：https://python.langchain.com/docs/integrations/vectorstores/oceanbase